# Trabalho Final - Aprendizado por Reforço

https://www.youtube.com/watch?app=desktop&v=bD6V3rcr_54&t=0s

https://www.jmlr.org/papers/v22/20-1364.html


https://gymnasium.farama.org/introduction/create_custom_env/

https://github.com/github0apurva/TabularData_Reinforcement

https://github.com/github0apurva/TabularData_Reinforcement/tree/main

https://stackoverflow.com/questions/58964267/how-to-create-an-openai-gym-observation-space-with-multiple-features

https://docs.ray.io/en/master/rllib/index.html

https://gymnasium.farama.org/api/spaces/utils/#gymnasium.spaces.utils.flatten_space


**https://medium.com/@tom.kaminski01/reinforcement-learning-for-f9a28632914f**

**https://gymnasium.farama.org/introduction/create_custom_env/**

In [88]:
import numpy as np
import pandas as pd
import random
from gym import Env
from gym.spaces import Discrete, Box
from sklearn.model_selection import train_test_split
import time

import tensorflow as tf

from stable_baselines3.dqn import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import PPO
from stable_baselines3.common import monitor 
from stable_baselines3.common import logger

### Separando os Conjuntos Teste x Treino

In [89]:
# Importando os dados do Excel
df_dataset = pd.read_excel('./DadosGolfitSoccer_2024_07_03.xlsx')
# Removendo colunas desnecessárias
df_dataset.drop(columns=['Codigo', 'Ano_Avaliacao', 'Nome', 'CategoriaEtaria', 'Ano_Nascimento','CLUBE', 'PosicaoJogo', 'YOYO_Percurso'], inplace=True)

In [90]:
df_dataset

,IdadeDecimal,Sucesso,Quartil_Nascimento,Dominância,Ambidestro,IdadeInicio,NivelDISPUTA,NivelVIToRIA,AtletanaFamilia,TempoPratica_anos,...,ForcaPreensao,Flexibilidade,CMJ,Agilidade,Drible,DeltaDrible,Vel10m,Vel20m,AvaliacaoTreinador,Intangiveis
0,14.658453,0,2,1,0,8.0,2,1.0,0,6.658453,...,22.0,27.0,35.9,7.525,10.560,3.035,1.891,3.296,3,31
1,13.913758,0,1,1,0,4.0,3,3.0,0,9.913758,...,25.0,17.5,37.6,7.559,10.276,2.717,1.874,3.213,3,26
2,16.928131,0,1,1,0,4.0,4,0.0,1,12.928131,...,38.0,21.5,28.1,7.480,10.220,2.740,1.685,2.950,5,38
3,13.774127,0,2,1,0,6.0,1,1.0,0,7.774127,...,14.0,26.0,26.7,8.733,12.700,3.967,1.949,3.463,2,18
4,15.956194,0,1,1,0,6.0,1,1.0,1,9.956194,...,32.0,35.5,34.7,8.870,11.960,3.090,1.939,3.317,4,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,15.219713,0,4,1,0,9.0,2,2.0,0,6.219713,...,31.1,23.0,25.6,7.487,10.481,2.994,1.774,3.073,3,35
123,15.006160,0,1,1,0,7.0,0,NaN,0,8.006160,...,38.1,21.0,42.2,7.598,10.449,2.851,1.486,3.073,3,24
124,15.134839,0,1,2,0,9.0,2,2.0,1,6.134839,...,36.4,25.0,36.1,8.043,11.118,3.075,1.641,2.689,2,32
125,16.183436,0,1,1,0,7.0,2,2.0,1,9.183436,...,24.4,19.2,36.4,6.988,9.641,2.653,1.640,2.913,3,35


In [91]:
# Separando os conjuntos de teste e treino
df_train, df_test = train_test_split(df_dataset, test_size=0.2)

# Retirando a coluna da classe - Sucesso
df_train_X = np.array(df_train.drop('Sucesso', axis=1))
# Convertendo a coluna para o tipo que será usado pelo modelo
df_train_y = np.array(df_train['Sucesso'])

# Testar o uso das dimensões expandidas
# df_train_X = np.expand_dims(df_train_X, 1)
# df_train_y = np.expand_dims(df_train_y, 1)

In [92]:
df_train_X

array([[16.81861739,  2.        ,  1.        , ...,  2.745     ,
         2.        , 17.        ],
       [17.00479124,  1.        ,  1.        , ...,  2.835     ,
         4.        , 39.        ],
       [13.79876797,  2.        ,  1.        , ...,         nan,
         3.        , 25.        ],
       ...,
       [14.57905544,  3.        ,  1.        , ...,  3.188     ,
         4.        , 45.        ],
       [16.56947296,  3.        ,  2.        , ...,  3.057     ,
         2.        , 23.        ],
       [13.83436003,  2.        ,  3.        , ...,  3.186     ,
         4.        , 41.        ]])

In [93]:
df_train_y

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0])

### Criando o ambiente para classificação

In [101]:
# Instanciando um ambiente a partir do ambiente fornecido pelo Gym
class PlayersEnv(Env):
    def __init__(self, majority_class=0, minority_class=1, row_per_episode=1, dataset=(df_train_X, df_train_y)):
        # As ações que o agente podem tomar são discretas, envolvendo classificar entre 2 classes
        self.action_space = Discrete(2)
        
        # O espaço de observações consiste nos jogadores que o agente deve classificar - 42 features para cada jogador
        obersevation = np.array([[np.finfo('float32').max] * 42], dtype=np.float32)
        self.observation_space = spaces.Box(-obersevation, obersevation, shape=(1, 42), dtype=np.float32)

        # Definindo qual as classe majoritária/minoritária do dataset - Para lidar com desbalanceamento dos dados
        self.majority_class = majority_class
        self.minority_class = minority_class

        # self.observations_space = Box(low=0, high=1, shape=())
        # Estado inicial em que o agente começa a interação
        self.state = 0
        # Tamanho do episódio == quantidade de instâncias (jogadores) a serem classificados
        self.episode_length = 0

        # Definição de parâmetros
        self.row_per_episode = row_per_episode
        self.step_count = 0
        self.x, self.y = dataset
        self.dataset_idx = 0
        

    # Função que especifica o que o agente fará a cada passo - Representa uma ação do agente
    # Recebe a ação tomada pelo agente como parâmetro
    def step(self, action):
        # Verificar se o agente classificou certo
        # Diferenciar recompensas para a classe maioritária e minoritária
        # Utilizar a fórmula definida pelos autores na metodologia do artigo

        # Flag que define se a tomada de decisão está finalizada
        done = False

        # Definição das recompensas caso o agente tome a decisão CORRETA
        if action == self.expected_action:
            match self.expected_action:
                # Caso o atleta seja da classe majoritária recebe uma recompensa menor
                case self.majority_class:
                    reward = 1
                # Caso o atleta seja da classe minoritária recebe uma recompensa maior
                case self.minority_class:
                    reward = 10

        # Definição das recompensas caso o agente tome a decisão INCORRETA
        if action != self.expected_action:
            match self.expected_action:
                # Caso o atleta seja da classe majoritária recebe uma recompensa menor
                case self.majority_class:
                    reward = -1
                # Caso o atleta seja da classe minoritária recebe uma recompensa maior
                case self.minority_class:
                    reward = -10

        # Passa para a próxima observação do dataset
        observation = self._next_observation()

        # Aumenta a contagem de decisões tomadas
        self.step_count += 1

        # Caso a quantidade de passos definidos por episódio tenha sido alcançada
        if self.step_count >= self.row_per_episode:
            done = True

        return obs, reward, done, {}  

    # Função que reseta o ambiente para treinar novamente
    def reset(self):
        # Zera a contagem de passos
        self.step_count = 0
        # Define a nova observação inicial
        observation = self._next_observation()
        # Retorna a observação inicial
        return observation

    # Subrotina que retorna a próxima observação para o agente
    def _next_observation(self):
        # Define um índice de forma aleatória para a próxima observação
        next_observation_idx = random.randint(0, len(self.x) - 1)
        # Define a ação esperada para a observação - Retorna o rótulo da observação
        self.expected_action = int(self.y[next_observation_idx])
        # Extrai a observação com o índice aleatório definido anteriormente
        observation = self.x[next_observation_idx]

        return observation
        

### Treinamento do modelo

In [102]:
# Configurando a pasta para logs
# logger.configure('./logs', ['stdout', 'tensorboard'])
# Instanciando o ambiente para treinamento
environment = PlayersEnv()
# Encapsulando o ambiente para logs
# environment = DummyVecEnv([lambda: monitor.Monitor(PlayersEnv(), logger.Logger('./logs',['stdout', 'tensorboard']).get_dir())])

# Inicializando o modelo PPO
model = PPO(
    'MlpPolicy', environment,
    verbose=1)

# Treinando o modelo
# model.learn(total_timesteps=int(1.2e4))
model.learn(total_timesteps=int(100))

# Medindo o tempo de treinamento
start_time = time.time()
print("Tempo de Treinamento PPO:", time.time() - start_time)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


ValueError: could not broadcast input array from shape (41,) into shape (1,42)